## Install the library

In [ ]:
!pip install geemap

## Import the Libraries

In [ ]:
import os
import ee
import geemap
import ipywidgets as widgets
import pandas as pd
import numpy as np

## Authenticate and Initialise with your account associated with Google Earth Engine

In [ ]:
ee.Authenticate()
ee.Initialize()

## Data Range

In [ ]:
start_date = '2019-01-01'
end_date = '2019-04-29'

In [ ]:
Map = geemap.Map()

geometry = ee.Geometry.Point([72.6026,23.0063])
image = ee.ImageCollection("COPERNICUS/S2_SR") \
            .filterBounds(geometry) \
            .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE',20)) \
            .first()
# Create an NDWI image, define visualization parameters and display.
ndwi = image.normalizedDifference(['B3', 'B8'])
# Mask the non-watery parts of the image, where NDWI < 0.4.
ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
ndwiMasked1= ndwiMasked.toInt()
# vectors = ndwiMasked1.reduceToVectors({
#   'scale': 30.0,
#   'geometryType': 'polygon',
#   'eightConnected': False,
#   'maxPixels':10000000
# })
vectors = ndwiMasked1.reduceToVectors(scale = 30.0, geometryType = 'polygon', eightConnected = False, maxPixels = 10000000, bestEffort=True)
# print(vectors)

In [ ]:
type(vectors)

ee.featurecollection.FeatureCollection

In [ ]:
sentinel = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(vectors) \
               .filterDate(start_date,end_date) \
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
               .median()


mndwi = sentinel.normalizedDifference(['B3','B11']).rename('mndwi')
mndwitr = mndwi.gt(0)
ndsi = sentinel.normalizedDifference(['B11','B12']).rename('ndsi')

In [ ]:
latlon = ee.Image.pixelLonLat().addBands(ndsi)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_ndsi = np.array((ee.Array(latlon.get("ndsi")).getInfo()))

In [ ]:
ndti = sentinel.normalizedDifference(['B4','B3']).rename('ndti')

ndci = sentinel.normalizedDifference(['B5','B4']).rename('ndci')


ph  = ee.Image(8.339).subtract(ee.Image(0.827).multiply(sentinel.select('B1').divide(sentinel.select('B8')))).rename('ph')

dissolvedoxygen  = ee.Image(-0.0167).multiply(sentinel.select('B8')).add(ee.Image(0.0067).multiply(sentinel.select('B9'))).add(ee.Image(0.0083).multiply(sentinel.select('B11'))).add(ee.Image(9.577)).rename('dissolvedoxygen')

In [ ]:
latlon = ee.Image.pixelLonLat().addBands(ndti)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100);
# get data into three different arrays
data_ndti = np.array((ee.Array(latlon.get("ndti")).getInfo()))

latlon = ee.Image.pixelLonLat().addBands(ndci)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100);
# get data into three different arrays
data_ndci = np.array((ee.Array(latlon.get("ndci")).getInfo()))

latlon = ee.Image.pixelLonLat().addBands(dissolvedoxygen)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_do = np.array((ee.Array(latlon.get("dissolvedoxygen")).getInfo()))


latlon = ee.Image.pixelLonLat().addBands(ph)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100);
# get data into three different arrays
data_ph = np.array((ee.Array(latlon.get("ph")).getInfo()))


In [ ]:
# print(image.bandNames().getInfo())

['Oa01_radiance', 'Oa02_radiance', 'Oa03_radiance', 'Oa04_radiance', 'Oa05_radiance', 'Oa06_radiance', 'Oa07_radiance', 'Oa08_radiance', 'Oa09_radiance', 'Oa10_radiance', 'Oa11_radiance', 'Oa12_radiance', 'Oa13_radiance', 'Oa14_radiance', 'Oa15_radiance', 'Oa16_radiance', 'Oa17_radiance', 'Oa18_radiance', 'Oa19_radiance', 'Oa20_radiance', 'Oa21_radiance', 'quality_flags']


In [ ]:
## Sentinel-3

starting = '2019-01-01'
ending = '2019-04-29'

data = ee.ImageCollection('COPERNICUS/S3/OLCI').filterDate(starting, ending).filterBounds(vectors)

rgb = data.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .median().multiply(ee.Image([0.00876539, 0.0123538, 0.0115198])).clip(vectors)
dm_2021_Jan_August_test = rgb.select('Oa08_radiance').divide(rgb.select('Oa04_radiance')).rename('dom')
suspended_matter_2021_Jan_August_test= rgb.select('Oa08_radiance').divide(rgb.select('Oa06_radiance')).rename('suspended_matter')

In [ ]:
latlon = ee.Image.pixelLonLat().addBands(dm_2021_Jan_August_test)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_dom_2021_Jan_August_test = np.array((ee.Array(latlon.get("dom")).getInfo()))

latlon = ee.Image.pixelLonLat().addBands(suspended_matter_2021_Jan_August_test)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_sm_2021_Jan_August_test= np.array((ee.Array(latlon.get("suspended_matter")).getInfo()))

In [ ]:
data_dom_2021_Jan_August_test

array([0.39272142, 0.37421201, 0.38044888, 0.37421201, 0.37421201,
       0.38044888, 0.37421201, 0.39272142, 0.39272142, 0.38044888,
       0.39272142, 0.39272142, 0.39272142, 0.39272142, 0.39272142,
       0.38044888, 0.38648775, 0.39272142, 0.38044888, 0.39272142,
       0.40499397, 0.38044888, 0.39856549, 0.38648775, 0.39856549,
       0.38044888, 0.38648775, 0.38648775, 0.39856549, 0.38648775,
       0.41064323, 0.41064323, 0.39800806, 0.39856549, 0.39856549,
       0.41064323, 0.3974839 , 0.3974839 , 0.39856549, 0.41064323,
       0.41064323, 0.3974839 , 0.3974839 , 0.39856549, 0.41064323,
       0.42272098, 0.40884059, 0.39856549, 0.40422693, 0.40350639,
       0.3974839 , 0.43155395, 0.43479872, 0.43479872, 0.41503514,
       0.43479872, 0.43312642, 0.44291064, 0.43989402, 0.42656389,
       0.44291064, 0.4496214 , 0.44291064, 0.465624  , 0.58989187,
       0.57899964, 0.55480707, 0.58735454, 0.57872128, 0.61221659,
       0.49234561, 0.6216485 , 0.5996479 , 0.58068513, 0.59123

In [ ]:
print(image.bandNames().getInfo())

['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10', 'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT']


In [ ]:
col = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
.filterDate('2019-01-01','2019-10-04') \
.filterBounds(vectors).median()

temp  = col.select('ST_B.*').multiply(0.00341802).add(149.0).subtract(273.15).rename('temp')



In [ ]:
print(col.bandNames().getInfo())

[]


In [ ]:
latlon = ee.Image.pixelLonLat().addBands(temp)


latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=vectors,
  scale=100,
  tileScale = 16);

data_lst = np.array((ee.Array(latlon.get("temp")).getInfo()))
# geemap.ee_to_numpy(dataset, region=geometry)

In [ ]:
data_lst

array([30.91702235, 30.55642124, 28.91064461, 27.9860702 , 27.37253561,
       26.57100992, 25.24823618, 25.36273985, 25.81391849, 25.73188601,
       24.58684931, 27.15549134, 32.16630866, 33.36603368, 28.43383082,
       27.44260502, 30.45558965, 33.13873535, 32.40557006, 33.21905882,
       25.6447265 , 27.50925641, 23.94767957, 23.72209025, 24.44500148,
       25.22772806, 26.34029357, 26.8068533 , 26.82736142, 26.81368934,
       26.90255786, 27.07004084, 25.26532628, 26.32491248, 26.80343528,
       26.96750024, 27.05636876, 27.44260502, 33.90437183, 32.15947262,
       32.45854937, 33.06695693, 34.60848395, 33.9163349 , 33.90095381,
       33.99494936, 34.24275581, 33.22076783, 32.10649331, 34.33504235,
       32.74053602, 32.93536316, 33.97444124, 32.92681811, 33.22931288,
       32.80206038, 32.94732623, 33.30280031, 32.48247551, 33.88044569,
       34.64608217, 32.30644748, 30.74270333, 30.2419634 , 31.69974893,
       32.53545482, 31.21780811, 31.99711667, 32.12016539, 32.30

# Finding water quality parameters

### Sentinel-2A

In [ ]:
#  ndsi = sentinel.normalizedDifference(['B11','B12']).rename('ndsi')
#   # ndsi2 = sentinel.normalizedDifference(['B11','B12']).rename('ndsi2').mask(mndwitr)
#   # Map.addLayer(ndsi2,{'min':0.1,'max':0.4,'palette':['cyan','orange','red']},'salinity')
#   ndti = sentinel.normalizedDifference(['B4','B3']).rename('ndti')
#   # ndti2 = sentinel.normalizedDifference(['B4','B3']).rename('ndti2').mask(mndwitr)
#   # Map.addLayer(ndti2,{'min':-1,'max':+1,'palette':['blue','pink','brown']},'turbidity')

#   ndci = sentinel.normalizedDifference(['B5','B4']).rename('ndci')
#   # ndci2 = sentinel.normalizedDifference(['B5','B4']).rename('ndci2').mask(mndwitr)
#   # Map.addLayer(ndci2,{'min':-1,'max':+1,'palette':['green','pink','brown']},'chlorophyll')


#   ph  = ee.Image(8.339).subtract(ee.Image(0.827).multiply(sentinel.select('B1').divide(sentinel.select('B8')))).rename('ph')
#   # ph2  = ee.Image(8.339).subtract(ee.Image(0.827).multiply(sentinel.select('B1').divide(sentinel.select('B8')))).rename('ph2').mask(mndwitr)
#   # Map.addLayer(ph2,{'min':0,'max':14,'palette':['red','yellow','cyan']},'ph')

#   dissolvedoxygen  = ee.Image(-0.0167).multiply(sentinel.select('B8')).add(ee.Image(0.0067).multiply(sentinel.select('B9'))).add(ee.Image(0.0083).multiply(sentinel.select('B11'))).add(ee.Image(9.577)).rename('dissolvedoxygen')
#   # dissolvedoxygen2  = ee.Image(-0.0167).multiply(sentinel.select('B8')).add(ee.Image(0.0067).multiply(sentinel.select('B9'))).add(ee.Image(0.0083).multiply(sentinel.select('B11'))).add(ee.Image(9.577)).rename('dissolvedoxygen2').mask(mndwitr)
#   # Map.addLayer(dissolvedoxygen2,{'min':6.5,'max':8,'palette':['red','green','blue']},'do')

In [ ]:
Map = geemap.Map()

# Kankaria Lake, Ahmedabad

geometry1 = ee.Geometry.Point([72.6026,23.0063])

geometry = ee.Geometry.Polygon([
    [72.5986408493042,23.006549566021803],
   [72.59902708740235,23.004890477468116],
   [72.60070078582764,23.003863412427236],
   [72.60040037841797,23.007142092704626],
   [72.60215990753174,23.006668071566512],
   [72.60173075408936,23.003784407100333],
    [72.60366194458008,23.00516699364359],
   [72.60374777526856,23.00686558057643],
    [72.6026748916626,23.00805062856477],
    [72.60082953186036,23.00880115357416],
    [72.59945624084473,23.00809012998513],
    [72.5986408493042,23.006549566021803]
])



sentinel = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(geometry) \
               .filterDate(start_date,end_date) \
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
               .median()


mndwi = sentinel.normalizedDifference(['B3','B11']).rename('mndwi')
mndwitr = mndwi.gt(0)
ndsi = sentinel.normalizedDifference(['B11','B12']).rename('ndsi')
ndti = sentinel.normalizedDifference(['B4','B3']).rename('ndti')

ndci = sentinel.normalizedDifference(['B5','B4']).rename('ndci')


ph  = ee.Image(8.339).subtract(ee.Image(0.827).multiply(sentinel.select('B1').divide(sentinel.select('B8')))).rename('ph')

dissolvedoxygen  = ee.Image(-0.0167).multiply(sentinel.select('B8')).add(ee.Image(0.0067).multiply(sentinel.select('B9'))).add(ee.Image(0.0083).multiply(sentinel.select('B11'))).add(ee.Image(9.577)).rename('dissolvedoxygen')


### Landsat-8

In [ ]:
col = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
.filterDate(start_date,end_date) \
.filterBounds(geometry).median()

temp  = col.select('ST_B.*').multiply(0.00341802).add(149.0).subtract(273.15).rename('temp')



### Sentinel-3

### Test Data

In [ ]:
# starting = '2021-01-01'
# ending = '2021-08-31'

starting = start_date
ending = end_date

data = ee.ImageCollection('COPERNICUS/S3/OLCI').filterDate(starting, ending).filterBounds(geometry)

rgb = data.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .median().multiply(ee.Image([0.00876539, 0.0123538, 0.0115198])).clip(geometry)
dm_2021_Jan_August_test = rgb.select('Oa08_radiance').divide(rgb.select('Oa04_radiance')).rename('dom')
suspended_matter_2021_Jan_August_test= rgb.select('Oa08_radiance').divide(rgb.select('Oa06_radiance')).rename('suspended_matter')


latlon = ee.Image.pixelLonLat().addBands(dm_2021_Jan_August_test)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=geometry,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_dom_2021_Jan_August_test = np.array((ee.Array(latlon.get("dom")).getInfo()))

latlon = ee.Image.pixelLonLat().addBands(suspended_matter_2021_Jan_August_test)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=geometry,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_sm_2021_Jan_August_test= np.array((ee.Array(latlon.get("suspended_matter")).getInfo()))

### Training Data

In [ ]:
start1 = '2021-01-01'
end1 = '2021-06-30'

In [ ]:
data = ee.ImageCollection('COPERNICUS/S3/OLCI').filterDate(start1, end1).filterBounds(geometry)

rgb = data.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .median().multiply(ee.Image([0.00876539, 0.0123538, 0.0115198])).clip(geometry)
dm_2021_Jan_June = rgb.select('Oa08_radiance').divide(rgb.select('Oa04_radiance')).rename('dom')
suspended_matter__2021_Jan_June = rgb.select('Oa08_radiance').divide(rgb.select('Oa06_radiance')).rename('suspended_matter')


In [ ]:
latlon = ee.Image.pixelLonLat().addBands(dm_2021_Jan_June)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=geometry,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_dom_2021_Jan_June = np.array((ee.Array(latlon.get("dom")).getInfo()))

In [ ]:
latlon = ee.Image.pixelLonLat().addBands(suspended_matter__2021_Jan_June)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=geometry,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_sm_2021_Jan_June= np.array((ee.Array(latlon.get("suspended_matter")).getInfo()))

In [ ]:
start2 = '2021-07-31'
end2 = '2021-10-31'

data = ee.ImageCollection('COPERNICUS/S3/OLCI').filterDate(start2, end2).filterBounds(geometry)

rgb = data.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .median().multiply(ee.Image([0.00876539, 0.0123538, 0.0115198])).clip(geometry)
dm_2021_July_Oct = rgb.select('Oa08_radiance').divide(rgb.select('Oa04_radiance')).rename('dom')
suspended_matter__2021_July_Oct = rgb.select('Oa08_radiance').divide(rgb.select('Oa06_radiance')).rename('suspended_matter')

latlon = ee.Image.pixelLonLat().addBands(dm_2021_July_Oct)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=geometry,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_dom_2021_July_Oct = np.array((ee.Array(latlon.get("dom")).getInfo()))

latlon = ee.Image.pixelLonLat().addBands(suspended_matter__2021_July_Oct)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=geometry,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_sm_2021_July_Oct= np.array((ee.Array(latlon.get("suspended_matter")).getInfo()))



start3 = '2021-11-30'
end3 = '2021-12-31'

data = ee.ImageCollection('COPERNICUS/S3/OLCI').filterDate(start3, end3).filterBounds(geometry)

rgb = data.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .median().multiply(ee.Image([0.00876539, 0.0123538, 0.0115198])).clip(geometry)
dm_2021_Nov_Dec = rgb.select('Oa08_radiance').divide(rgb.select('Oa04_radiance')).rename('dom')
suspended_matter__2021_Nov_Dec = rgb.select('Oa08_radiance').divide(rgb.select('Oa06_radiance')).rename('suspended_matter')

latlon = ee.Image.pixelLonLat().addBands(dm_2021_Nov_Dec)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=geometry,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_dom_2021_Nov_Dec = np.array((ee.Array(latlon.get("dom")).getInfo()))

latlon = ee.Image.pixelLonLat().addBands(suspended_matter__2021_Nov_Dec)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=geometry,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_sm_2021_Nov_Dec = np.array((ee.Array(latlon.get("suspended_matter")).getInfo()))

### Temperature Calculation

In [ ]:
latlon = ee.Image.pixelLonLat().addBands(temp)


latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=geometry,
  scale=100);

data_lst = np.array((ee.Array(latlon.get("temp")).getInfo()))
# geemap.ee_to_numpy(dataset, region=geometry)

### Turbidity Calculation

In [ ]:
latlon = ee.Image.pixelLonLat().addBands(ndti)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=geometry,
  scale=100);
# get data into three different arrays
data_ndti = np.array((ee.Array(latlon.get("ndti")).getInfo()))

### Salinity Calculation

In [ ]:
latlon = ee.Image.pixelLonLat().addBands(ndsi)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=geometry,
  scale=100);
# get data into three different arrays
data_ndsi = np.array((ee.Array(latlon.get("ndsi")).getInfo()))

### Chlorophyll Calculation

In [ ]:
latlon = ee.Image.pixelLonLat().addBands(ndci)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=geometry,
  scale=100);
# get data into three different arrays
data_ndci = np.array((ee.Array(latlon.get("ndci")).getInfo()))

### Dissolved Oxygen Calculation

In [ ]:
latlon = ee.Image.pixelLonLat().addBands(dissolvedoxygen)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=geometry,
  scale=100,
  tileScale = 16);
# get data into three different arrays
data_do = np.array((ee.Array(latlon.get("dissolvedoxygen")).getInfo()))

### pH Calculation

In [ ]:
latlon = ee.Image.pixelLonLat().addBands(ph)
# apply reducer to list
latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=geometry,
  scale=100);
# get data into three different arrays
data_ph = np.array((ee.Array(latlon.get("ph")).getInfo()))

## Concating dissolved organic matter and suspended matter values

In [ ]:
dm = [pd.DataFrame(data_dom_2021_Jan_June, columns = ['Dissolved Organic Matter']),\
      pd.DataFrame(data_dom_2021_July_Oct, columns = ['Dissolved Organic Matter']),\
      pd.DataFrame(data_dom_2021_Nov_Dec, columns = ['Dissolved Organic Matter'])]

sm = [pd.DataFrame(data_sm_2021_Jan_June, columns = ['Suspended Matter']),\
      pd.DataFrame(data_sm_2021_July_Oct, columns = ['Suspended Matter']),\
      pd.DataFrame(data_sm_2021_Nov_Dec, columns = ['Suspended Matter'])]
data_dom = pd.concat(dm, sort=False, ignore_index=True)
data_sm = pd.concat(sm, sort=False, ignore_index=True)

## Creating Dataframe

In [ ]:
df = pd.concat([pd.DataFrame(data_do, columns = ['Dissolved Oxygen']),\
           pd.DataFrame(data_ndsi, columns = ['Salinity']),\
           pd.DataFrame(data_lst, columns = ['Temperature']),\
           pd.DataFrame(data_ph, columns = ['pH']),\
           pd.DataFrame(data_ndti, columns = ['Turbidity']),\
           pd.DataFrame(data_dom, columns = ['Dissolved Organic Matter']),\
           pd.DataFrame(data_sm, columns = ['Suspended Matter']),\
           pd.DataFrame(data_ndci, columns = ['Chlorophyll'])], axis=1, sort=False)

In [ ]:
df

## Converting to CSV files

In [ ]:
df.to_csv('Data_2021_Tappar_Lake',index=False)

## Test Data

In [ ]:
df2 = pd.concat([pd.DataFrame(data_do, columns = ['Dissolved Oxygen']),\
           pd.DataFrame(data_ndsi, columns = ['Salinity']),\
           pd.DataFrame(data_lst, columns = ['Temperature']),\
           pd.DataFrame(data_ph, columns = ['pH']),\
           pd.DataFrame(data_ndti, columns = ['Turbidity']),\
           pd.DataFrame(data_dom_2021_Jan_August_test, columns = ['Dissolved Organic Matter']),\
           pd.DataFrame(data_sm_2021_Jan_August_test, columns = ['Suspended Matter']),\
           pd.DataFrame(data_ndci, columns = ['Chlorophyll'])], axis=1, sort=False)
df2.head()
df2.to_csv('Test_Data_2021_Jan_August_Kankaria_Lake',index=False)